**Imports**

In [ ]:
from Assistant import Retrieve_Context, Ask_Assistant

import lancedb
#import openai
from dotenv import load_dotenv
import groq

#### Using LLM as a Judge  
> For evaluation of the RAG response, an effective method is to use another LLM instance to evaluate the response provided  
> This can be made by evaluating the RAG response with the criteria such as Groundedness, Completeness, Relevance etc

In [ ]:
# Initialise an client object with API key
load_dotenv ()
Eval_Client = groq.Groq ()

In [ ]:
# Create a Lance DB Vector Base
DB = lancedb.connect ('Quick_Ref')

**Sample Questions**  
Prepare a list of sample questions based on the subject that is preserved in the Knowledge base  
This list will be used to generate Response from the RAG pipeline  

In [ ]:
Table_Name = 'temp'

Queries = ["How different the software development be in Quantum computers?",
           "What is the processor in Quantum Computer?",
           "Better computing performance"]


**Evaluation**  
> For a set of questions, generate RAG response  
> For the same set of questions, retrieve the content using the same retrieval mechanism (consistency)  
> Based on the question, context and generated answer, use another LLM to make evaluation on the required criteria  
> The criteria can also talk about generic qualititative aspects

In [ ]:
Judge_Instruction = """ You are an evaluator who can check an answer provided by LLM.
                    You are given with Context information, User Query and LLM Answer.
                    You will have to evalute the answer with following "criteria".
                    1. "Groundedness" : Whether the answer is based on details in context
                    2. "Completeness" : Whether the answer is complete to answer the question
                    3. "Relevance" : Whether the answer is relevant to the question

                    Provide your evaluation in (Good / Moderate / Bad) scale in format. 
                    [{"Criteria" : Evaluation}]
                    """

for Query in Queries :

    # Fetch the context, with same method as it is given for Generation
    Context = Retrieve_Context (Query, 'temp')

    # Get the answer from the Generation Assistant
    Answer = Ask_Assistant ("", Query=Query, table_name='temp')

    messages=[
    {
        "role": "system",
        "content": Judge_Instruction,
    },
    {
        "role": "user",
        "content": "Context : \n"+str(Context),
    },    
    {
        "role": "user",
        "content": "User Query : \n"+Query,
    },    
    {
        "role": "user",
        "content": "LLM Answer : \n"+Answer,
    }, 
    ]

    completion = Eval_Client.chat.completions.create(
        messages=messages,    
        # model="llama-3.3-70b-versatile",
        model="openai/gpt-oss-120b",
        stop=None,
    )
    print ("Question :\n", Query, "Answer : \n",Answer, "---\n",completion.choices[0].message.content)

